<a href="https://colab.research.google.com/github/Mondkuchen/idp_LiteratureResearch_Tool/blob/main/scripts/train/KeyBartAdapter_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.22.1 datasets==2.5.2 evaluate==0.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import numpy as np


# 1. Dataset
from datasets import load_dataset
dataset = load_dataset("Adapting/abstract-keyphrases")

# 2. Model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Adapting/KeyBartAdapter")

model = AutoModelForSeq2SeqLM.from_pretrained("Adapting/KeyBartAdapter", revision = '9c3ed39c6ed5c7e141363e892d77cf8f589d5999')


# 3. preprocess dataset
dataset = dataset.shuffle()
def preprocess_function(examples):
    inputs = examples['Abstract']
    targets = examples['Keywords']
    model_inputs = tokenizer(inputs, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

# 4. evaluation metrics
def compute_metrics(eval_preds):
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    if isinstance(preds, tuple):
        preds = preds[0]
    print(preds.shape)
    if len(preds.shape) == 3:
        preds = preds.argmax(axis=-1)
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [a.strip().split(';') for a in decoded_preds]
    decoded_labels = [a.strip().split(';') for a in decoded_labels]


    precs, recalls, f_scores = [], [], []
    num_match, num_pred, num_gold = [], [], []
    for pred, label in zip(decoded_preds, decoded_labels):
        pred_set = set(pred)
        label_set = set(label)
        match_set = label_set.intersection(pred_set)
        p = float(len(match_set)) / float(len(pred_set)) if len(pred_set) > 0 else 0.0
        r = float(len(match_set)) / float(len(label_set)) if len(label_set) > 0 else 0.0
        f1 = float(2 * (p * r)) / (p + r) if (p + r) > 0 else 0.0
        precs.append(p)
        recalls.append(r)
        f_scores.append(f1)
        num_match.append(len(match_set))
        num_pred.append(len(pred_set))
        num_gold.append(len(label_set))
        
        # print(f'raw_PRED: {raw_pred}')
        print(f'PRED: num={len(pred_set)} - {pred_set}')
        print(f'GT: num={len(label_set)} - {label_set}')
        print(f'p={p}, r={r}, f1={f1}')
        print('-' * 20)

    result = {
        'precision@M': np.mean(precs) * 100.0,
        'recall@M': np.mean(recalls) * 100.0,
        'fscore@M': np.mean(f_scores) * 100.0,
        'num_match': np.mean(num_match),
        'num_pred': np.mean(num_pred),
        'num_gold': np.mean(num_gold),
    }

    result = {k: round(v, 2) for k, v in result.items()}
    return result

# 5. train
from transformers import DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

model_name = 'KeyBartAdapter'
num_epoch = 30

args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_epoch,
    logging_steps=4,
    load_best_model_at_end=True,
    metric_for_best_model='fscore@M',
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # speeds up training on modern GPUs.
    # eval_accumulation_steps=10,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of the model checkpoint at Adapting/KeyBartAdapter were not used when initializing BartForConditionalGeneration: ['model.decoder.decoder.layers.11.encoder_attn.v_proj.weight', 'model.decoder.decoder.layers.5.self_attn_layer_norm.weight', 'model.decoder.decoder.layers.11.self_attn.out_proj.weight', 'model.decoder.decoder.layers.2.encoder_attn.q_proj.weight', 'model.decoder.decoder.layers.9.fc1.weight', 'model.decoder.decoder.layers.8.self_attn.out_proj.weight', 'model.decoder.decoder.layers.3.self_attn.out_proj.weight', 'model.decoder.adapters.7.up_proj.weight', 'model.decoder.decoder.layers.0.self_attn.q_proj.bias', 'model.decoder.decoder.layers.1.final_layer_norm.bias', 'model.decoder.decoder.layers.5.self_attn.q_proj.weight', 'model.decoder.decoder.layers.11.fc2.weight', 'model.decoder.decoder.layers.3.self_attn.q_proj.weight', 'model.decoder.decoder.layers.6.encoder_attn.out_proj.bias', 'model.decoder.decoder.layers.3.encoder_attn.v_proj.bias', 'model.decoder.decoder.la

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3543: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 50
  Num Epochs = 30
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accu

Epoch,Training Loss,Validation Loss,Precision@m,Recall@m,Fscore@m,Num Match,Num Pred,Num Gold
1,1.484700,0.823063,55.000000,37.330000,43.590000,1.220000,2.240000,3.280000
2,0.836800,0.575592,63.070000,62.600000,61.430000,1.960000,3.240000,3.280000
3,0.664900,0.431628,78.070000,68.970000,71.810000,2.180000,2.860000,3.280000
4,0.390500,0.280106,90.670000,69.630000,77.340000,2.160000,2.420000,3.280000
5,0.344900,0.316271,92.170000,70.200000,78.500000,2.240000,2.460000,3.280000
6,0.355400,0.223609,90.830000,80.430000,84.580000,2.560000,2.860000,3.280000
7,0.343300,0.320883,88.000000,80.430000,83.370000,2.540000,2.920000,3.280000
8,0.308200,0.186449,91.500000,83.500000,86.710000,2.640000,2.920000,3.280000
9,0.156900,0.259272,91.430000,86.770000,88.500000,2.740000,3.040000,3.280000
10,0.130000,0.191015,89.430000,85.700000,87.120000,2.700000,3.080000,3.280000


...

Epoch,Training Loss,Validation Loss,Precision@m,Recall@m,Fscore@m,Num Match,Num Pred,Num Gold
1,1.484700,0.823063,55.000000,37.330000,43.590000,1.220000,2.240000,3.280000
2,0.836800,0.575592,63.070000,62.600000,61.430000,1.960000,3.240000,3.280000
3,0.664900,0.431628,78.070000,68.970000,71.810000,2.180000,2.860000,3.280000
4,0.390500,0.280106,90.670000,69.630000,77.340000,2.160000,2.420000,3.280000
5,0.344900,0.316271,92.170000,70.200000,78.500000,2.240000,2.460000,3.280000
6,0.355400,0.223609,90.830000,80.430000,84.580000,2.560000,2.860000,3.280000
7,0.343300,0.320883,88.000000,80.430000,83.370000,2.540000,2.920000,3.280000
8,0.308200,0.186449,91.500000,83.500000,86.710000,2.640000,2.920000,3.280000
9,0.156900,0.259272,91.430000,86.770000,88.500000,2.740000,3.040000,3.280000
10,0.130000,0.191015,89.430000,85.700000,87.120000,2.700000,3.080000,3.280000


***** Running Evaluation *****
  Num examples = 50
  Batch size = 4


(50, 20)
PRED: num=4 - {'transfer learning', 'transformer', 'natural language processing', 'fine-tuning'}
GT: num=4 - {'transfer learning', 'transformer', 'natural language processing', 'fine-tuning'}
p=1.0, r=1.0, f1=1.0
--------------------
PRED: num=3 - {'hypergraph attention network', 'recommender system', 'session-based recommendation system'}
GT: num=3 - {'hypergraph attention network', 'recommender system', 'session-based recommendation system'}
p=1.0, r=1.0, f1=1.0
--------------------
PRED: num=3 - {'descriptive text caption', 'generating audio sample', 'auto-regressive generative model'}
GT: num=3 - {'descriptive text caption', 'generating audio sample', 'auto-regressive generative model'}
p=1.0, r=1.0, f1=1.0
--------------------
PRED: num=3 - {'human motion generation', 'motion diffusion model', 'diffusion model'}
GT: num=3 - {'human motion generation', 'motion diffusion model', 'diffusion model'}
p=1.0, r=1.0, f1=1.0
--------------------
PRED: num=3 - {'spiking neural netw

Saving model checkpoint to KeyBartAdapter/checkpoint-390
Configuration saved in KeyBartAdapter/checkpoint-390/config.json
Model weights saved in KeyBartAdapter/checkpoint-390/pytorch_model.bin
tokenizer config file saved in KeyBartAdapter/checkpoint-390/tokenizer_config.json
Special tokens file saved in KeyBartAdapter/checkpoint-390/special_tokens_map.json
Deleting older checkpoint [KeyBartAdapter/checkpoint-364] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from KeyBartAdapter/checkpoint-208 (score: 90.32).


TrainOutput(global_step=390, training_loss=0.24197826716953363, metrics={'train_runtime': 2544.9523, 'train_samples_per_second': 0.589, 'train_steps_per_second': 0.153, 'total_flos': 961529921028096.0, 'train_loss': 0.24197826716953363, 'epoch': 30.0})

In [ ]:
abstract = '''Large text-to-image models achieved a remarkable leap in the evolution of AI, enabling high-quality and diverse synthesis of images from a given text prompt. However, these models lack the ability to mimic the appearance of subjects in a given reference set and synthesize novel renditions of them in different contexts. In this work, we present a new approach for "personalization" of text-to-image diffusion models (specializing them to users' needs). Given as input just a few images of a subject, we fine-tune a pretrained text-to-image model (Imagen, although our method is not limited to a specific model) such that it learns to bind a unique identifier with that specific subject. Once the subject is embedded in the output domain of the model, the unique identifier can then be used to synthesize fully-novel photorealistic images of the subject contextualized in different scenes. By leveraging the semantic prior embedded in the model with a new autogenous class-specific prior preservation loss, our technique enables synthesizing the subject in diverse scenes, poses, views, and lighting conditions that do not appear in the reference images. We apply our technique to several previously-unassailable tasks, including subject recontextualization, text-guided view synthesis, appearance modification, and artistic rendering (all while preserving the subject's key features). Project page: https://dreambooth.github.io/'''

In [ ]:
from transformers import Text2TextGenerationPipeline
pipe = Text2TextGenerationPipeline(model=model,tokenizer=tokenizer,device= 0)


pipe(abstract)

[{'generated_text': 'text-to-image diffusion model;subject recontextualization'}]

In [ ]:
# # 6. push to hub
# commit_msg = f'{model_name}_{num_epoch}'
# tokenizer.push_to_hub(commit_message=commit_msg, repo_id=model_name )
# model.push_to_hub(commit_message=commit_msg, repo_id=model_name)